# Train and Apply Models

In [1]:
from ML.model_training import (
    omit_patient_video,
    single_user_split,
    train_lstm,
    train_random_forest,
    build_lstm_sequences
)
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    classification_report,
)
import numpy as np
import pandas as pd
from ML import utils
import sys
import random
from itertools import product

In [2]:
# remove_list = [5, 18]
remove_list = []

def balance(X, y, seed=5):
    c = y.value_counts()
    if c.get("high", 0) == c.get("low", 0):
        return X.reset_index(drop=True), y.reset_index(drop=True)
    maj = c.idxmax()
    m = c.min()
    keep = y[y != maj].index.union(y[y == maj].sample(m, random_state=seed).index)
    return X.loc[keep].reset_index(drop=True), y.loc[keep].reset_index(drop=True)

# subjects = [i for i in range(0, 23)]
subjects = [15]

## LSTM CV Optimizer

This is currently running LOSO CV, by removing 18 videos (all) from each user.

In [3]:
param_grid = {
    "lr": [0.0001],
    "epochs": [100],
    "units": [512],
    "batch_size": [128],
    "timesteps": [None]
}

best_params = None
best_mean_acc = -float("inf")

print("Starting global hyperparameter search...\n")

for lr, epochs, units, batch_size, timesteps in product(
    param_grid["lr"],
    param_grid["epochs"],
    param_grid["units"],
    param_grid["batch_size"],
    param_grid["timesteps"],
):
    combo_accs = []
    for i in subjects:
        relabel = False
        while True:
            while True:
                X_train_df, X_test_df, arousal_train, arousal_test = omit_patient_video(
                    target="arousal",
                    selected_user=i,
                    trials=18,
                    # holdout_videos=[2, 10, 15],
                    exclude_users=remove_list,
                    filename="datasets/features_table.csv",
                    relabel=relabel,
                )

                features = utils.filter_features(
                    X_train_df.columns,
                    remove_bands=["gamma"],
                )
                features = [c for c in features if c in X_train_df.columns]
                features.remove("arousal")
                features.remove("valence")
                features.remove("patient_index")
                features.remove("video_index")

                # Build sequence-level data (trials = (patient, video))
                X_train_seq, y_train_seq = build_lstm_sequences(
                    X_train_df,
                    features,
                    target_col="arousal",
                    thresh=3.8,
                    fixed_T=timesteps,
                )
                X_test_seq, y_test_seq = build_lstm_sequences(
                    X_test_df,
                    features,
                    target_col="arousal",
                    thresh=3.8,
                    fixed_T=timesteps
                )

                minority = (y_train_seq == 1.0).sum() < (y_train_seq == 0.0).sum()
                maj_label = 0.0 if minority else 1.0
                min_label = 1.0 - maj_label

                idx_maj = np.where(y_train_seq == maj_label)[0]
                idx_min = np.where(y_train_seq == min_label)[0]

                if len(idx_min) == 0:
                    # no samples of one class, redo split
                    print("No minority class in this split, re-drawing...")
                    continue

                # reps = int(np.ceil(len(idx_maj) / len(idx_min)))
                # idx_min_upsampled = np.tile(idx_min, reps)[: len(idx_maj)]

                # idx_balanced = np.concatenate([idx_maj, idx_min_upsampled])
                # np.random.shuffle(idx_balanced)

                # X_train_bal = X_train_seq[idx_balanced]
                # y_train_bal = y_train_seq[idx_balanced]

                print("y_train counts:", np.bincount(y_train_seq.astype(int)))
                print("y_test counts:", np.bincount(y_test_seq.astype(int)))

                break

            lstm, X_test_eval, y_test_eval = train_lstm(
                X_train_seq,
                X_test_seq,
                y_train_seq,
                y_test_seq,
                lr=lr,
                epochs=epochs,
                units=units,
                batch_size=batch_size,
                dropout=0.4,
                recurrent_dropout=0.2,
                bidirectional=True,
            )
            y_prob = lstm.predict(X_test_eval).ravel()
            arousal_pred = (y_prob >= 0.5).astype(int)

            acc = accuracy_score(y_test_eval, arousal_pred)
            print(acc)
            relabel = False
            break

        combo_accs.append(float(acc))

        print("\nConfusion Matrix (subject):")
        print(confusion_matrix(y_test_eval, arousal_pred))
    mean_acc = float(np.mean(combo_accs))
    print(
        f"Params lr={lr}, epochs={epochs}, units={units}, batch_size={batch_size}, timesteps={timesteps} "
        f"-> mean acc across subjects = {mean_acc:.4f}"
    )

    if mean_acc > best_mean_acc:
        best_lstm = lstm
        best_mean_acc = mean_acc
        best_params = {
            "lr": lr,
            "epochs": epochs,
            "units": units,
            "batch_size": batch_size,
            "timesteps": timesteps
        }


print("\nBest universal hyperparameters:")
print(best_params)
print(f"Mean accuracy across subjects (tuning): {best_mean_acc:.4f}\n")

Starting global hyperparameter search...

Held-out patient: 15 | Held-out (patient, video) trials: [(15, 0), (15, 1), (15, 2), (15, 3), (15, 4), (15, 5), (15, 6), (15, 7), (15, 8), (15, 9), (15, 10), (15, 11), (15, 12), (15, 13), (15, 14), (15, 15), (15, 16), (15, 17)] | Excluded users: []
y_train counts: [223 173]
y_test counts: [10  8]
X_train_arr shape: (396, 196, 476)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 815ms/step
0.6666666666666666

Confusion Matrix (subject):
[[10  0]
 [ 6  2]]
Params lr=0.0001, epochs=100, units=512, batch_size=128, timesteps=None -> mean acc across subjects = 0.6667

Best universal hyperparameters:
{'lr': 0.0001, 'epochs': 100, 'units': 512, 'batch_size': 128, 'timesteps': None}
Mean accuracy across subjects (tuning): 0.6667



### Fine-tune for User

In [4]:
print(best_lstm.layers)

[<InputLayer name=input_layer, built=True>, <Bidirectional name=bidirectional, built=True>, <Dense name=dense, built=True>, <Dropout name=dropout, built=True>, <Dense name=dense_1, built=True>]


In [ ]:
from tensorflow.keras.models import clone_model
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from sklearn.metrics import confusion_matrix, accuracy_score
import tensorflow as tf
import numpy as np
from itertools import product

param_grid = {
    "lr": [0.0001],
    "epochs": [50],
    "batch_size": [32],
    "patience": [3],
    "layers": [1, 2],
    "units": [64, 128],
}

T_SIZE = 196
N_RUNS = 1
SELECTED_USER = 15
TRIALS = [i for i in range(0, 18, 2)]

best_params = None
best_mean_acc = -np.inf

print("Starting per-user fine-tuning hyperparameter search...\n")

for lr, epochs, batch_size, patience, layers, units in product(
    param_grid["lr"],
    param_grid["epochs"],
    param_grid["batch_size"],
    param_grid["patience"],
    param_grid["layers"],
    param_grid["units"],
):

    combo_accs = []

    print(
        f"Testing combo: lr={lr}, batch_size={batch_size}, "
        f"epochs={epochs}, patience={patience}, layers={layers}, units={units}"
    )

    for run in range(N_RUNS):
        (
            X_user_train_df,
            X_user_test_df,
            arousal_user_train,
            arousal_user_test,
        ) = single_user_split(
            target="arousal",
            selected_user=SELECTED_USER,
            holdout_videos=TRIALS,
            k_holdouts=9,
            filename="datasets/features_table.csv",
        )

        X_user_train_seq, y_user_train_seq = build_lstm_sequences(
            X_user_train_df,
            feature_cols=features,
            target_col="arousal",
            thresh=3.8,
            fixed_T=T_SIZE,
        )
        X_user_test_seq, y_user_test_seq = build_lstm_sequences(
            X_user_test_df,
            feature_cols=features,
            target_col="arousal",
            thresh=3.8,
            fixed_T=T_SIZE,
        )

        base_model = clone_model(best_lstm)
        base_model.build(best_lstm.input_shape)
        base_model.set_weights(best_lstm.get_weights())

        for layer in base_model.layers[:-layers]:
            layer.trainable = False

        inputs = tf.keras.Input(shape=best_lstm.input_shape[1:])
        x = base_model(inputs, training=True)
        x = tf.keras.layers.Dense(units, activation="relu")(x)
        outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
        user_model = tf.keras.Model(inputs, outputs)

        opt = optimizers.Adam(learning_rate=lr)
        user_model.compile(optimizer=opt,
                           loss="binary_crossentropy",
                           metrics=["accuracy"])

        es = callbacks.EarlyStopping(
            monitor="val_loss",
            patience=patience,
            restore_best_weights=True,
        )

        user_model.fit(
            X_user_train_seq,
            y_user_train_seq,
            validation_split=0.2,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[es],
            verbose=0,
        )

        y_prob = user_model.predict(X_user_train_seq).ravel()
        print(y_prob)
        arousal_pred = (y_prob >= 0.5).astype(int)

        acc = accuracy_score(y_user_train_seq, arousal_pred)
        cm = confusion_matrix(y_user_train_seq, arousal_pred)
        print("Confusion matrix:\n", cm)

        combo_accs.append(acc)

    mean_acc = np.mean(combo_accs)
    std_acc = np.std(combo_accs)
    print(f"  -> mean acc over {N_RUNS} runs: {mean_acc:.4f} (std={std_acc:.4f})\n")

    if mean_acc > best_mean_acc:
        best_mean_acc = mean_acc
        best_params = {
            "lr": lr,
            "batch_size": batch_size,
            "epochs": epochs,
            "patience": patience,
            "units": units,
            "layers": layers,
        }

print(f"Best mean test accuracy: {best_mean_acc:.4f}\n")
print(best_params)


Starting per-user fine-tuning hyperparameter search...

Testing combo: lr=0.0001, batch_size=32, epochs=50, patience=3, layers=1, units=64
